# Dolphin Gradio App
Interactive web interface for Dolphin document parsing

In [1]:
#!pip3.10 install hf_xet

In [2]:
import gradio as gr
import os
from PIL import Image
import json
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import warnings
warnings.filterwarnings("ignore")

c:\Users\omont\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\omont\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\omont\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\omont\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# Initialize Qwen2.5-VL model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize variables
model = None
processor = None

try:
    # Load Qwen2.5-VL-3B-Instruct model with simpler configuration
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2.5-VL-3B-Instruct", 
        torch_dtype="auto",
        device_map="auto",
        trust_remote_code=True
    )
    processor = AutoProcessor.from_pretrained(
        "Qwen/Qwen2.5-VL-3B-Instruct",
        trust_remote_code=True
    )
    
    print(f"Gradio version: {gr.__version__}")
    print("✅ Qwen2.5-VL-3B-Instruct model loaded successfully!")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Error details:", str(e))
    # Set to None explicitly if loading fails
    model = None
    processor = None

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Using device: cuda


You are using a model of type qwen2_5_vl to instantiate a model of type qwen2_vl. This is not supported for all configurations of models and can yield errors.
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards:   0%|          | 0/2 [00:06<?, ?it/s]


❌ Error loading model: Trying to set a tensor of shape torch.Size([2048]) in "bias" (which has shape torch.Size([1280])), this looks incorrect.
Error details: Trying to set a tensor of shape torch.Size([2048]) in "bias" (which has shape torch.Size([1280])), this looks incorrect.


In [4]:
def process_document(image):
    """
    Process document image with Qwen2.5-VL
    """
    if image is None:
        return "Please upload an image"
    
    # Check if model and processor are loaded
    if model is None or processor is None:
        return "❌ Model or processor not loaded. Please check the model initialization above."
    
    try:
        # Simple approach without qwen_vl_utils
        # Convert PIL image to format expected by processor
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": "Analiza esta imagen de documento. Describe qué ves, extrae todo el texto que puedas leer, identifica tablas, fórmulas, diagramas o cualquier elemento estructural. Proporciona una descripción detallada del contenido."}
                ]
            }
        ]
        
        # Apply chat template
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Process inputs
        inputs = processor(text=text, images=[image], return_tensors="pt")
        inputs = inputs.to(device)
        
        # Generate response
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                temperature=0.7,
                pad_token_id=processor.tokenizer.eos_token_id
            )
        
        # Decode response
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        
        output_text = processor.batch_decode(
            generated_ids_trimmed, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False
        )
        
        return output_text[0] if output_text else "No response generated"
        
    except Exception as e:
        return f"Error processing image: {str(e)}"

In [5]:
# Create Gradio interface
demo = gr.Interface(
    fn=process_document,
    inputs=gr.Image(type="pil", label="Upload Document Image"),
    outputs=gr.Textbox(label="Qwen2.5-VL Analysis Results", lines=10),
    title="Qwen2.5-VL-3B-Instruct Document Analyzer",
    description="Upload a document image to analyze with Qwen2.5-VL-3B-Instruct. The model will extract text, identify structures, and provide detailed analysis.",
    examples=[
        ["demo/page_imgs/page_1.jpeg"] if os.path.exists("demo/page_imgs/page_1.jpeg") else None
    ]
)

if __name__ == "__main__":
    demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
